In [37]:
import pandas as pd
from backtesting import Backtest, Strategy
from datetime import datetime
from data_loader import PykrxDataLoader

In [38]:
# RSI 계산 함수 (DataFrame 입력 버전)
def calculate_rsi(df, window=14):
    
    """
    DataFrame에 RSI 컬럼 추가
    <파라미터>
    df: 종가(Close) 컬럼을 포함한 DataFrame
    window: 계산 기간 (기본 14일)
    <반환값>
    RSI 값이 추가된 DataFrame
    """

    # 전날과 가격차이 계산
    delta = df['Close'].diff()

    # 상승폭(up)과 하락폭(down) 계산
    up = delta.where(delta > 0, 0)
    down = -delta.where(delta < 0, 0)
    
    # 평균 상승폭(AU)와 평균 하락폭(AD) 계산 (이동평균)
    avg_up = up.rolling(window=window).mean()
    avg_down = down.rolling(window=window).mean()
    
    # 상대강도(RS) 계산
    rs = avg_up / avg_down
    
    # RSI 계산
    df['RSI'] = 100 - (100 / (1 + rs))
    return df

In [ ]:
# RSI 전략 클래스
class RSIStrategy(Strategy):
    rsi_window = 14
    oversold = 30
    overbought = 70

    def init(self):
        # DataFrame을 직접 전달하도록 수정
        self.rsi = self.I(
            lambda close_series: calculate_rsi(
                pd.DataFrame({'Close': close_series}),  # 시리즈 → DF 변환
                self.rsi_window
            ),
            self.data.Close  # 입력 데이터 (시리즈)
        )


    def next(self):
        current_rsi = self.rsi[-1]
        
        if not self.position and current_rsi < self.oversold:
            self.buy()
        elif self.position and current_rsi > self.overbought:
            self.position.close() 

In [43]:
# 데이터 로드 (PykrxDataLoader 사용)
fromdate = '2021-05-01'
todate = datetime.now().strftime("%Y-%m-%d")
ticker_list = ['005930']

loader = PykrxDataLoader(
    fromdate = fromdate,
    todate = todate,
    market = 'KOSPI'
)
df = loader.load_stock_data(ticker_list=ticker_list, freq='d', delay=1)


# 백테스트 실행
bt = Backtest(
    df, 
    RSIStrategy, 
    cash=10_000_000, 
    commission=0.00015, 
    exclusive_orders=True,
)

stats = bt.run()
print(stats)

bt.plot()

Start                     2021-05-03 00:00:00
End                       2025-05-07 00:00:00
Duration                   1465 days 00:00:00
Exposure Time [%]                    59.91862
Equity Final [$]                  10018286.44
Equity Peak [$]                   12420346.31
Commissions [$]                       31151.4
Return [%]                            0.18286
Buy & Hold Return [%]               -31.74404
Return (Ann.) [%]                     0.04685
Volatility (Ann.) [%]                20.26571
CAGR [%]                              0.03143
Sharpe Ratio                          0.00231
Sortino Ratio                         0.00362
Calmar Ratio                          0.00125
Alpha [%]                            19.89969
Beta                                  0.62112
Max. Drawdown [%]                     -37.569
Avg. Drawdown [%]                    -9.02159
Max. Drawdown Duration      898 days 00:00:00
Avg. Drawdown Duration      128 days 00:00:00
# Trades                          

GridPlot(id='p8012', ...)

In [36]:
# 전략 인스턴스 추출
strategy_instance = stats._strategy

trades_df = stats['_trades']
trades_df['EntryRSI'] = trades_df['EntryBar'].apply(
    lambda x: float(strategy_instance.rsi[x]) if x < len(strategy_instance.rsi) else None
)
trades_df['ExitRSI'] = trades_df['ExitBar'].apply(
    lambda x: float(strategy_instance.rsi[x]) if x < len(strategy_instance.rsi) else None
)
print(trades_df[['EntryBar','ExitBar','EntryRSI','ExitRSI']])

    EntryBar  ExitBar EntryRSI ExitRSI
0         36      139     None    None
1        139      140     None    None
2        140      149     None    None
3        149      150     None    None
4        150      151     None    None
..       ...      ...      ...     ...
82       953      954     None    None
83       954      955     None    None
84       955      956     None    None
85       956      957     None    None
86       957      958     None    None

[87 rows x 4 columns]
